# Setup 

In [1]:
!pip install kaggle_environments

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.3 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

# Download precompiled .so
!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.3.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .
!pip install reinforcement_learning_keras

     |████████████████████████████████| 1.9 MB 13.9 MB/s 
     |████████████████████████████████| 63 kB 792 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/cu

 # All the imports 

In [2]:
import matplotlib.pyplot as plt
import pprint
import glob 
import imageio
import pathlib
import numpy as np
from typing import Tuple
from tqdm import tqdm
import os
import sys
from IPython.display import Image
# from kaggle_football.viz import generate_gif, plot_smm_obs

# Gym Interface

In [3]:
"""import gym
import gfootball  # Required as envs registered on import

simple_env = gym.make("GFootball-11_vs_11_kaggle-simple115v2-v0") #LIST WITH
#pixels_env = gym.make("GFootball-11_vs_11_kaggle-Pixels-v0") #ena me pixels kanonika
#smm_env = gym.make("GFootball-11_vs_11_kaggle-SMM-v0") #DOTS AS PLAYERS - IMAGES

print(f"simple115v2:\n {simple_env.__str__()}\n")
#print(f"Pixels:\n {pixels_env.__str__()}\n")
#print(f"SMM:\n {smm_env.__str__()}\n")"""

'import gym\nimport gfootball  # Required as envs registered on import\n\nsimple_env = gym.make("GFootball-11_vs_11_kaggle-simple115v2-v0") #LIST WITH\n#pixels_env = gym.make("GFootball-11_vs_11_kaggle-Pixels-v0") #ena me pixels kanonika\n#smm_env = gym.make("GFootball-11_vs_11_kaggle-SMM-v0") #DOTS AS PLAYERS - IMAGES\n\nprint(f"simple115v2:\n {simple_env.__str__()}\n")\n#print(f"Pixels:\n {pixels_env.__str__()}\n")\n#print(f"SMM:\n {smm_env.__str__()}\n")'

# Environment

In [4]:
import gym
import gfootball
from gfootball.env.football_env import FootballEnv
from kaggle_environments.envs.football.helpers import *

env_name = "GFootballBase-v0"
gym.envs.register(id=env_name,
                  entry_point="gfootball.env.football_env:FootballEnv",
                  max_episode_steps=10000)

In [5]:
from gfootball.env.config import Config
#Observation wrappers convert raw observation to different form
base_env = gym.make(env_name, config=Config())#ftiaxno enviroment me gym.make  me to onoma tou env GFootballBase-v0
simple_env = gym.make("GFootball-11_vs_11_kaggle-simple115v2-v0")#simplified representation of a game state encoded with 115 floats

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


# Observation Space

In [20]:
obs = base_env.reset()
print(obs[0])
#print(obs) #obs.shape
pprint.pprint(obs)


{'ball_owned_team': -1, 'right_team_direction': array([[-0.,  0.],
       [-0.,  0.],
       [-0.,  0.],
       [-0.,  0.],
       [-0.,  0.],
       [-0.,  0.],
       [-0.,  0.],
       [-0.,  0.],
       [-0.,  0.],
       [-0.,  0.],
       [-0.,  0.]]), 'ball_owned_player': -1, 'steps_left': 3001, 'right_team': array([[ 1.01102936,  0.        ],
       [-0.        , -0.02032536],
       [-0.        ,  0.02032536],
       [ 0.4266544 ,  0.19894461],
       [ 0.50551468,  0.06459399],
       [ 0.50551468, -0.06459298],
       [ 0.4266544 , -0.19894461],
       [ 0.18624374,  0.10739919],
       [ 0.27052519,  0.        ],
       [ 0.18624374, -0.10739919],
       [ 0.01011029,  0.2196155 ]]), 'right_team_roles': array([0, 7, 9, 2, 1, 1, 3, 5, 5, 5, 6]), 'right_team_active': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'ball_direction': array([ 0.        , -0.        ,  0.00616395]), 'score': [0, 0], 'left_team_active': array([ True,  T

# Conver it to asingle  array 

In [68]:
obs = simple_env.reset()

print(obs.shape) #(115,)
print(simple_env.observation_space.shape) 
pprint.pprint(obs) # numpy.ndarray 
print(len(obs))


(115,)
(115,)
array([-1.0110294 , -0.        ,  0.        ,  0.02032536,  0.        ,
       -0.02032536, -0.4266544 , -0.19894461, -0.5055147 , -0.06459399,
       -0.5055147 ,  0.06459298, -0.4266544 ,  0.19894461, -0.18624374,
       -0.10739919, -0.2705252 , -0.        , -0.18624374,  0.10739919,
       -0.01011029, -0.2196155 ,  0.        , -0.        ,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        , -0.        ,
        0.        , -0.        ,  0.        , -0.        ,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        , -0.        ,
        0.        , -0.        ,  0.        , -0.        ,  1.0110294 ,
        0.        ,  0.05055147,  0.        ,  0.01011029, -0.21961753,
        0.4266544 ,  0.19894461,  0.5055147 ,  0.06459399,  0.5055147 ,
       -0.06459298,  0.4266544 , -0.19894461,  0.18624374,  0.10739919,
        0.2705252 ,  0.        ,  0.18624374, -0.10739919,  0.01011029,
        0.2196155 , -0.        ,  0.        , -0. 

In [86]:
print(obs[88],obs[89],obs[90]) #BALL POSITION [X,Y,Z]
print(obs[91],obs[92],obs[93]) #BALL DIRECTION [X,Y,Z]
print(obs[94],obs[95],obs[96])#BALL OWNERSHIP [NOONE,LEFT,RIGHT]
print(obs[97],obs[98],obs[99],obs[100],obs[101],obs[102],obs[103],obs[104],obs[105],obs[106],obs[107])#which player is active - current agent 
print(obs[108],obs[109],obs[110],obs[111],obs[112],obs[113],obs[114])#Game mode[Normal, KickOff, GoalKick, FreeKick, Corner, ThrowIn, Penalty]

0.0 -0.0 0.11061639
-0.0 0.0 0.006163953
1.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
1.0 0.0 0.0 0.0 0.0 0.0 0.0


In [74]:
act =5
new_obs,rew,done,info = simple_env.step(act)
new_obs,rew,done,info = simple_env.step(act)
new_obs,rew,done,info = simple_env.step(act)
print(new_obs[88],new_obs[89],new_obs[90]) # POSITION OF THE BALL[X,Y,Z]

0.011312323 0.0037471985 0.28165093


##  Number of Actions

In [ ]:
simple_env.action_space.n

19

# Make an Action take new Observation_State

In [ ]:
new_act = simple_env.action_space.sample()
print("discrete action" , new_act)
#print(Action.Slide)
new_obs,rew,done,info = simple_env.step(new_act)
print(new_obs,"\t",rew,"\t",done,"\t",info) 

discrete action 5
[-1.0110294e+00 -0.0000000e+00  1.2774976e-10  1.9232830e-02
  1.7134001e-03 -2.1353513e-02 -4.2472520e-01 -1.9796792e-01
 -5.0551468e-01 -6.4593993e-02 -5.0551468e-01  6.4592980e-02
 -4.2660055e-01  1.9891958e-01 -1.8624374e-01 -1.0739919e-01
 -2.6954600e-01  1.2148561e-06 -1.8624374e-01  1.0739919e-01
 -9.8830406e-03 -2.1814944e-01  0.0000000e+00 -0.0000000e+00
  2.5157859e-10 -2.3336501e-03  2.7455224e-03 -1.5774269e-03
  3.0461480e-03  1.6057320e-03  0.0000000e+00 -0.0000000e+00
  0.0000000e+00 -0.0000000e+00  5.3854549e-04 -2.5028246e-04
  0.0000000e+00 -0.0000000e+00  3.2360062e-03  5.4484722e-06
  0.0000000e+00 -0.0000000e+00  3.7283573e-04  2.5281389e-03
  1.0110294e+00  0.0000000e+00  4.8872512e-02 -1.3186933e-11
  1.0943406e-02 -2.1889924e-01  4.2665440e-01  1.9894461e-01
  5.0551468e-01  6.4593993e-02  5.0551468e-01 -6.4592980e-02
  4.2665440e-01 -1.9894461e-01  1.8624374e-01  1.0739919e-01
  2.7052519e-01  0.0000000e+00  1.8624374e-01 -1.0739919e-01
  1.01

# Visual representation

In [ ]:
obs = smm_env.reset()


print(f"Shape: {obs.shape}\n")
print('Team 1:')
plt.imshow(obs[:, :, 0])
plt.show()

print('Team 2:')
plt.imshow(obs[:, :, 1])
plt.show()

print('Position of the ball:')
plt.imshow(obs[:, :, 2])
plt.show()

print('Position of active player:')
plt.imshow(obs[:, :, 3])
plt.show()

# Now, you can run it:

In [ ]:
    def _play_episode(self, max_episode_steps: int = 500,
                      training: bool = False, render: bool = True) -> Tuple[float, int]:
        """
        Play a single episode and return the total reward.
        :param max_episode_steps: Max steps before stopping, overrides any time limit set by Gym.
        :param training: Bool to indicate whether or not to use this experience to update the model.
        :param render: Bool to indicate whether or not to call env.render() each training step.
        :return: The total real reward for the episode.
        """
        self.env._max_episode_steps = max_episode_steps
        obs = self.env.reset()
        total_reward = 0
        for frame in range(max_episode_steps):
            action = self.get_action(obs, training=training)
            prev_obs = obs
            obs, reward, done, info = self.env.step(action)
            total_reward += reward

            if render:
                self.env.render()

            if training:
                self.update_model(s=prev_obs, a=action, r=reward, d=done, s_=obs)

            if done:
                break

        return total_reward, frame

    

# Simple training Deep Q Network - keras

In [ ]:
import gym
from reinforcement_learning_keras.agents.components.history.training_history import TrainingHistory
from reinforcement_learning_keras.agents.q_learning.exploration.epsilon_greedy import EpsilonGreedy
from reinforcement_learning_keras.agents.q_learning.linear_q_agent import LinearQAgent
from sklearn.exceptions import DataConversionWarning

import warnings
if __name__ == "__main__":
  #ORIZO TON AGENT KAI TO PERIBALLON EKPAIDEYSIS
    agent = LinearQAgent(name="linear_q",
                         env_spec="GFootball-11_vs_11_kaggle-simple115v2-v0",
                         eps=EpsilonGreedy(eps_initial=0.9, decay=0.001, eps_min=0.01, decay_schedule='linear'),
                         training_history=TrainingHistory(agent_name='linear_q', plotting_on=True, plot_every=10, 
                                                          rolling_average=1))
    #DEN EMFANIZEI TA WARININGS

    with warnings.catch_warnings():
        warnings.simplefilter('ignore', DataConversionWarning)


        agent.train(verbose=True, render=False,
                    n_episodes=20, max_episode_steps=2000)  #typose gia 20 episodia ,20 bimata to kathe episodio . nai verbose , oxi se grafika

In [ ]:
#pixels_env.render()
#obs = pixels_env.reset()

In [ ]:
import gym
import gfootball  # Required as envs registered on import

simple_env = gym.make("GFootball-11_vs_11_kaggle-simple115v2-v0") # me ta noumera 
pixels_env = gym.make("GFootball-11_vs_11_kaggle-Pixels-v0")# me tous anthropous 
smm_env = gym.make("GFootball-11_vs_11_kaggle-SMM-v0")# me tis koukides 

print(f"simple115v2:\n {simple_env.__str__()}\n")
print(f"Pixels:\n {pixels_env.__str__()}\n")
print(f"SMM:\n {smm_env.__str__()}\n")